# Automação

In [1]:
#Bibliotecas utilizadas
import pandas as pd #Manipulação de arquivos
from bs4 import BeautifulSoup #Varredura no linkedin
from requests import get #Requisição
from sqlalchemy import create_engine# Criar conexão com o banco de dados SQLite


In [2]:
#Dados para pesquisar
nome_vaga = input('Digite o nome da vaga: ')
nome_cidade = input('Digite a cidade: ')
nome_estado = input('Digite o estado: ')
nome_pais = input('Digite o pais: ')
numero_paginas = int(input('Quantas páginas quer buscar: '))
filtro = int(input('''
                   1 -Vagas em 24h 
                   2 -A qualquer momento
                   '''))
if filtro == 1:
    filtro_tempo = 'f_TPR=r86400&'
else:
    filtro_tempo = ''

In [3]:
#Tratamento dos dados
vaga_tratada = nome_vaga.replace(' ', "%20")
cidade_tratada = nome_cidade.replace(' ', "%20")
estado_tratado = nome_estado.replace(' ', "%20")
pais_tratado = nome_pais.replace(' ', "%20")


In [4]:
#Função para buscar vagas
def buscar_vagas():
    #Lista para armazenar vagas
    vagas_filtradas = []

    #Loop para percorrer páginas
    for pagina in range(1, numero_paginas):
        #URL Base do Linkedin
        url = f'''https://www.linkedin.com/jobs/search?keywords={vaga_tratada}&location={cidade_tratada}%2C%20{estado_tratado}%2C%20{pais_tratado}&geoId=104746682&{filtro_tempo}position=1&pageNum={pagina}'''
        
        #Baixar o conteúdo da página do linkedin
        resposta = get(url).text

        #Normalizando o html
        resposta_tratada = BeautifulSoup(resposta, 'html.parser')

        #Procurando vagas por tag e classe HTML
        vagas = resposta_tratada.find_all('div', class_ = 'base-card')

        #Loop para percorrer a página, filtrar nome da vaga, empresa, estado e data de publicação e link da vaga e salvar em um JSON
        for vaga in vagas:

            vagas_filtradas.append({
                    "Vaga": vaga.find("h3").text.strip(),
                    "Empresa": vaga.find("h4").text.strip(),
                    "Estado": vaga.find("span", class_="job-search-card__location").text.strip(),
                    "Data de Publicação": vaga.find("time").text.strip(),
                    "Link da vaga": vaga.a["href"]
                })
    return vagas_filtradas

In [8]:
salvar_vagas = int(input('''
                         1 - Salvar em csv
                         2 - Salvar no SQLite
                         '''))

#Salvando em um DataFrame
df = pd.DataFrame(buscar_vagas())

if salvar_vagas == 1:
    #Salvando em arquivo csv
    df.to_csv('Vagas Pesquisadas', index=False)
    
elif salvar_vagas == 2:
    # Criar engine do SQLite
    engine = create_engine('sqlite:///vagas.db')

    # Inserir o DataFrame na tabela 'vagas'
    df.to_sql('vagas', engine, index=False, if_exists='replace')

In [6]:

#Quantidade de vagas por empresa
qtd = df['Empresa'].value_counts()

In [7]:
import plotly.express as plt

plt.bar(qtd, x=qtd.index, y=qtd.values)